In [3]:
from typing import List

import numpy as np
import sounddevice as sd

from pyquist import AudioProcessor, AudioBuffer

class AdditiveSynthesizer(AudioProcessor):
    def __init__(self, *, pitches: List[float], amplitudes: List[float]):
        self._pitches = pitches
        self._amplitudes = np.array(amplitudes)

    def prepare_to_play(self, sample_rate: float, block_size: int):
        super().prepare_to_play(sample_rate, block_size)
        self._time = 0.0
        self._block_length_seconds = block_size / sample_rate
        self._frequencies = np.array([440.0 * 2**((p - 69) / 12.0) for p in self._pitches])

    def process_block(self, buffer: AudioBuffer):
        t = self._time + (np.arange(self.block_size) / self.sample_rate)
        buffer[:] = np.sum(
            self._amplitudes[:, np.newaxis] * np.sin(2.0 * np.pi * t[np.newaxis] * self._frequencies[:, np.newaxis]), axis=0)
        self._time += self._block_length_seconds


# TODO: AudioContext? Base on Tone.JS?

SAMPLE_RATE = 44100
BLOCK_SIZE = 512

synth = AdditiveSynthesizer(pitches=[60, 64, 67], amplitudes=[0.5, 0.4, 0.3])
synth.prepare_to_play(SAMPLE_RATE, BLOCK_SIZE)

def audio_callback(outdata, frames, time, status):
    buffer = AudioBuffer(num_channels=1, num_samples=BLOCK_SIZE, array=outdata.swapaxes(0, 1))
    synth.process_block(buffer)

# Start the audio stream with the callback function
with sd.OutputStream(samplerate=SAMPLE_RATE, blocksize=BLOCK_SIZE, channels=1, callback=audio_callback):
    sd.sleep(1000)  # Run the stream for 10 seconds (adjust as needed)

In [ ]:
from pyquist import Audio
from pyquist.notebook import play

url = "https://github.com/librosa/data/raw/refs/heads/main/audio/198-209-0000.hq.ogg"

audio = Audio.from_url(url)
audio.normalize(peak_dbfs=-18.0)
print(audio.peak_amplitude)
play(audio, normalize=True)
play(audio, normalize=False)